In [1]:
import pickle

bios = pickle.load(open("corpus1000.pickle", "rb"))

In [82]:
bios[47][0][1]

[('Summary',
  'Henri-Louis Bergson (; 18 October 1859 – 4 January 1941) was a major French philosopher, influential especially in the first half of the 20th century. Bergson convinced many thinkers that the processes of immediate experience and intuition are more significant than abstract rationalism and science for understanding reality.\n\nHe was awarded the 1927 Nobel Prize in Literature "in recognition of his rich and vitalizing ideas and the brilliant skill with which they have been presented". In 1930 France awarded him its highest honour, the Grand-Croix de la Legion d\'honneur.\n\n'),
 ('Philosophy',
  'Bergson rejected what he saw as the overly mechanistic predominant view of causality (as expressed in, say, finalism). He argued that we must allow space for free will to unfold in an autonomous and unpredictable fashion. While Kant saw free will as something beyond time and space and therefore ultimately a matter of faith, Bergson attempted to redefine the modern conceptions o

In [3]:
import sys

def progress(i, end_val, bar_length=50):
    percent = float(i) / end_val
    hashes = '#' * int(round(percent * bar_length))
    spaces = ' ' * (bar_length - len(hashes))
    sys.stdout.write("\r{0} / {1} Percent: [{2}] {3}%".format(i, end_val, hashes + spaces, int(round(percent * 100))))
    sys.stdout.flush()

In [4]:
import os, lda, numpy as np
from nltk import word_tokenize, sent_tokenize

all_tokens = set()
paragraph_tokens = list()
biographies_paragraphs = list()
biographies_paragraphs_tokens = list()

bios_subset = bios[:1000]

i = 0
for bio in bios_subset:
    biography_paragraphs = list()
    biography_paragraphs_tokens = list()
    for section in bio[0][1]:
        raw_section_text = section[1]
        raw_section_paragraphs_text = raw_section_text.split('\n')
        for raw_paragraph_text in raw_section_paragraphs_text:
            stripped_paragraph_text = raw_paragraph_text.strip('\n')
            if len(stripped_paragraph_text) == 0:
                continue

            tokens = word_tokenize(stripped_paragraph_text)
#             sentences = sent_tokenize(raw_text)
            all_tokens |= set(tokens)
            paragraph_tokens.append(tokens)
        
            biography_paragraphs.append(stripped_paragraph_text)
            biography_paragraphs_tokens.append(tokens)

#             biography_sentences.append(sentences)
#             biography_sentences_tokens.append(list(word_tokenize(sentence) for sentence in sentences))
    biographies_paragraphs.append(biography_paragraphs)
    biographies_paragraphs_tokens.append(biography_paragraphs_tokens)

    i += 1
    progress(i, len(bios_subset))

1000 / 1000 Percent: [##################################################] 100%

//anaconda/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [70]:
for i in range(len(biographies_paragraphs_tokens)):
    if len(biographies_paragraphs_tokens[i]) > 30:
        print(i, len(biographies_paragraphs_tokens[i]))

47 32
59 40
70 34
72 75
76 33
81 32
93 33
94 42
97 81
98 108
104 67
115 52
119 39
122 32
125 33
133 37
139 75
145 69
165 32
173 120
176 45
181 57
187 41
204 34
209 46
214 37
217 77
225 46
233 58
245 34
252 35
255 47
259 45
265 39
266 99
269 39
276 75
288 38
299 39
311 47
316 65
326 57
328 39
330 38
333 39
338 40
342 44
359 67
384 35
386 82
406 38
414 34
416 50
430 37
478 42
479 168
480 43
493 181
494 44
505 42
517 71
527 55
542 41
544 50
545 31
546 34
550 33
564 39
571 36
573 68
575 33
599 51
610 53
619 55
625 78
635 76
644 137
645 31
675 41
676 63
683 36
691 74
699 53
710 32
728 55
748 66
764 38
781 44
783 61
787 33
794 42
797 39
810 48
813 62
823 54
828 137
833 53
834 36
839 74
843 41
865 62
867 76
868 61
871 59
877 41
878 34
887 68
893 98
904 46
909 55
911 50
914 92
929 58
938 42
987 42
990 38


In [6]:
all_tokens_list = list(all_tokens)
number_of_paragraphs = len(paragraph_tokens)
number_of_tokens = len(all_tokens_list)
paragraphs_bow = np.zeros([number_of_paragraphs, number_of_tokens], dtype = np.int)

tokens_indices_dict = dict()
for i in range(number_of_tokens):
    tokens_indices_dict[all_tokens_list[i]] = i

for i in range(number_of_paragraphs):
    for j in range(len(paragraph_tokens[i])):
        paragraphs_bow[i][tokens_indices_dict[paragraph_tokens[i][j]]] += 1
        
    progress(i + 1, number_of_paragraphs)

14035 / 14035 Percent: [##################################################] 100%

In [7]:
len(all_tokens_list)

58979

In [8]:
vocab = all_tokens_list
paragraphs_bow.shape

(14035, 58979)

In [9]:
number_of_topics = 30

model = lda.LDA(n_topics=number_of_topics, n_iter=1500, random_state=1)
model.fit(paragraphs_bow)  # model.fit_transform(X) is also available

In [11]:
topic_word = model.topic_word_  # model.components_ also works
n_top_words = 20

topics_words = list()

word_freqs = dict()

for i, topic_dist in enumerate(topic_word):
    # topic_words: words sorted by relevance to topic in descending order
    topic_words = list(np.array(vocab)[np.argsort(topic_dist)[::-1]])#[:10]#[:-(n_top_words+1):-1]
    #print(topic_words)
    topics_words.append(topic_words)
    print('Topic {}: {}'.format(i, ' '.join(topic_words[:15])))
    for word in topic_words[:20]:
        if word not in word_freqs:
            word_freqs[word] = 1
        else:
            word_freqs[word] += 1
            
topic_words_indices_dicts = list()
for topic_words in topics_words:
    topic_words_indices_dict = dict()
    for i in range(len(topic_words)):
        topic_words_indices_dict[topic_words[i]] = i
    topic_words_indices_dicts.append(topic_words_indices_dict)

Topic 0: , . the and '' of `` in a on music 's with by The
Topic 1: the to , . was that a his and had he of by for in
Topic 2: , ) ( . was and a is He born an known ; of American
Topic 3: ) ( '' * : = , director ISBN & - C der und {
Topic 4: the of , . in and to was Pope his He on Rome as he
Topic 5: , '' the . in and for of a film as 's ''The on with
Topic 6: the of , . in and a to on theory for ) 's his He
Topic 7: , the of . and in a his with art 's work by works painting
Topic 8: , . '' : * ; of and [ ] The Press ISBN London ``
Topic 9: the of and , . to a in 's The with for as were which
Topic 10: the , of . and in to his was a as by Roman He against
Topic 11: the of , in . and at University a was he He In to from
Topic 12: , the . in and a for on of to is In has an 's
Topic 13: , '' of . the and in 's `` his a published by ''The for
Topic 14: '' ) ( * The `` ''The ; : 's ** A with ! ISBN
Topic 15: '' , ) ( * by di R Italian Naples La ''La Teatro e dir
Topic 16: , the . `` '' that

In [12]:
def calculate_word_topic_id(word, topics_words):
    if word in word_freqs and word_freqs[word] > 4:
        return -1
    
    topic_id = -1
    min_topic_index = number_of_tokens + 1
    for current_topic_id in range(number_of_topics):
        index = topic_words_indices_dicts[current_topic_id][word]
        if index < min_topic_index:
            topic_id = current_topic_id
            min_topic_index = index
            
    return topic_id

In [13]:
vocabulary_words_indices_dict = dict()
for i in range(len(vocab)):
    vocabulary_words_indices_dict[vocab[i]] = i

In [14]:
def calculate_vocabulary_topic_ids(vocabulary, topics_words):
    topic_ids = list()
    i = 0
    for word in vocab:
        topic_ids.append(calculate_word_topic_id(word, topics_words))
        i += 1
        progress(i, len(vocab))
        
    return topic_ids

In [15]:
vocabulary_topic_ids = calculate_vocabulary_topic_ids(vocab, topics_words)

57081 / 58979 Percent: [################################################  ] 97%

In [16]:
def get_vocabulary_word_topic_id(vocabulary, vocabulary_topic_ids, word):
    try:
        return vocabulary_topic_ids[vocabulary_words_indices_dict[word]]
    except ValueError:
        # If word is not in the vocabulary (e.g. 'h.50')
        return -1

In [17]:
get_vocabulary_word_topic_id(vocab, vocabulary_topic_ids, "George")

20

In [18]:
def get_paragraph_topic_ids(paragraph, topics_words):
    topic_ids = list()
    for word in paragraph:
        topic_ids.append(get_vocabulary_word_topic_id(vocab, vocabulary_topic_ids, word))
        
    return topic_ids

In [19]:
def get_paragraphs_topic_ids(paragraphs, topic_words):
    topic_ids = list()
    for paragraph in paragraphs:
        topic_ids.append(get_paragraph_topic_ids(paragraph, topics_words))
        
    return topic_ids

In [20]:
def get_biographies_paragraphs_topic_ids(biographies_paragraphs_tokens, topic_words):
    biographies_paragraphs_topic_ids = list()
    i = 0
    for biography in biographies_paragraphs_tokens:
        biography_paragraphs_topic_ids = get_paragraphs_topic_ids(biography, topic_words)
        biographies_paragraphs_topic_ids.append(biography_paragraphs_topic_ids)
        i += 1
        progress(i, len(biographies_paragraphs_tokens))
    
    return biographies_paragraphs_topic_ids

biographies_paragraphs_topic_ids = get_biographies_paragraphs_topic_ids(biographies_paragraphs_tokens, topic_words)

1000 / 1000 Percent: [##################################################] 100%

In [21]:
biographies_paragraphs_topic_ids[0][0]

[11,
 11,
 -1,
 2,
 14,
 -1,
 -1,
 -1,
 11,
 -1,
 27,
 -1,
 11,
 -1,
 -1,
 11,
 11,
 -1,
 11,
 -1,
 29,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 11,
 -1,
 11,
 -1,
 27,
 -1,
 6,
 -1,
 12,
 -1]

In [22]:
biographies_paragraphs_tokens[2][0]

['Aimoin',
 '(',
 'circa',
 '960',
 'c.',
 '1010',
 ')',
 ',',
 'French',
 'chronicler',
 ',',
 'was',
 'born',
 'at',
 'Villefranche-de-Longchat',
 'about',
 '960',
 ',',
 'and',
 'in',
 'early',
 'life',
 'entered',
 'the',
 'monastery',
 'of',
 'Fleury',
 ',',
 'where',
 'he',
 'became',
 'a',
 'monk',
 'and',
 'passed',
 'the',
 'greater',
 'part',
 'of',
 'his',
 'life',
 '.']

In [23]:
def calculate_paragraph_topics_frequency(paragraph_topic_ids, number_of_topics):
    topics_frequencies = np.zeros(number_of_topics)
    for topic_id in paragraph_topic_ids:
        if (topic_id >= 0):
            topics_frequencies[topic_id] += 1
    
    return topics_frequencies

In [24]:
def calculate_paragraphs_topics_frequency(paragraphs_topic_ids, number_of_topics):
    paragraphs_topics_frequencies = list()
    for paragraph in paragraphs_topic_ids:
        paragraphs_topics_frequencies.append(calculate_paragraph_topics_frequency(paragraph, number_of_topics))
        
    return paragraphs_topics_frequencies

In [25]:
# def calculate_biography_paragraphs_topics_frequencies(biography_paragraphs_topic_ids, number_of_topics):
#     biography_paragraphs_topics_frequencies = list()
#     for paragraphs in biography_paragraphs_topic_ids:
#         biography_paragraphs_topics_frequencies.append(calculate_paragraphs_topics_frequency(paragraphs, number_of_topics))
        
#     return biography_paragraphs_topics_frequencies

In [26]:
def calculate_biographies_paragraphs_topics_frequencies(biographies_paragraphs_topic_ids, number_of_topics):
    biographies_paragraphs_topics_frequencies = list()
    i = 0
    for biography_paragraphs_topic_ids in biographies_paragraphs_topic_ids:
        biographies_paragraphs_topics_frequencies.append(calculate_paragraphs_topics_frequency(biography_paragraphs_topic_ids, number_of_topics))
        i += 1
        progress(i, len(biographies_paragraphs_topic_ids))
        
    return biographies_paragraphs_topics_frequencies

In [27]:
biographies_paragraphs_topics_frequencies = calculate_biographies_paragraphs_topics_frequencies(biographies_paragraphs_topic_ids, number_of_topics)

1000 / 1000 Percent: [##################################################] 100%

In [28]:
biographies_paragraphs_topics_frequencies[2][0]

array([ 0.,  0.,  1.,  0.,  2.,  0.,  0.,  0.,  0.,  2.,  1.,  0.,  1.,
        0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,
        0.,  7.,  5.,  0.])

In [29]:
def calculate_coherence_score(paragraphs_topics_frequencies, number_of_topics, position, window_size):
    pre_topics_frequencies = np.zeros(number_of_topics)
    for i in range(window_size):
        pre_topics_frequencies += paragraphs_topics_frequencies[position - i]
        
    post_topics_frequencies = np.zeros(number_of_topics)
    for i in range(window_size):
        post_topics_frequencies += paragraphs_topics_frequencies[position + 1 + i]
        
    dot_product = np.dot(pre_topics_frequencies, post_topics_frequencies)
    pre_norm = np.linalg.norm(pre_topics_frequencies)
    post_norm = np.linalg.norm(post_topics_frequencies)
    
    cosine_similarity = dot_product / (pre_norm * post_norm)
    
    return cosine_similarity

In [30]:
calculate_coherence_score(biographies_paragraphs_topics_frequencies[1], number_of_topics, 10, 3)

0.87692689757987219

In [31]:
window_size = 3

def calculate_coherence_scores(paragraphs_topics_frequencies, number_of_topics, window_size):
    coherence_scores = list()
    for i in range(len(paragraphs_topics_frequencies) - 2 * window_size):
        score = calculate_coherence_score(paragraphs_topics_frequencies, number_of_topics, window_size + i - 1, window_size)
        coherence_scores.append(score)
        
    return coherence_scores

In [32]:
def calculate_depth_scores(coherence_scores):
    depth_scores = list()
    for i in range(len(coherence_scores)):
        hl = coherence_scores[i]
        hr = coherence_scores[i]
        for j in range(i):
            if coherence_scores[i - j - 1] > hl:
                hl = coherence_scores[i - j - 1]
            else:
                break
        
        for j in range(i + 1, len(coherence_scores)):
            if coherence_scores[j] > hr:
                hr = coherence_scores[j]
            else:
                break
                
        depth_score = 0.5 * (hl + hr - 2 * coherence_scores[i])
        depth_scores.append(depth_score)
        
    return depth_scores

In [71]:
bio_index = 47
coherence_scores = calculate_coherence_scores(biographies_paragraphs_topics_frequencies[bio_index], number_of_topics, window_size)

In [72]:
depth_scores = calculate_depth_scores(coherence_scores)
len(coherence_scores)

26

In [73]:
len(biographies_paragraphs_topics_frequencies[bio_index][1])

30

In [74]:
mean = np.mean(depth_scores)

In [75]:
std = np.std(depth_scores)

In [76]:
mean - std / 2

0.0071024121374120841

In [81]:
for i in range(len(depth_scores)):
    print(depth_scores[i] , "\t" , biographies_paragraphs[bio_index][i + window_size][:50])

0.0199062171616 	 Bergson considers the appearance of novelty as a r
0.0114871185325 	 Criticizing Kant's theory of knowledge exposed in 
0.0 	 ''The Creative Evolution'' in particular attempted
0.0300743073988 	 Henri Bergsons ''Lebensphilosophie'' (Philosophy o
0.000710596862901 	 Bergson regarded planning beforehand for the futur
0.0 	 Duration (philosophy)
0.00300854435808 	 The foundation of Henri Bergsons philosophy, his t
0.041334680721 	 Kant believed that free will could only exist outs
0.0 	 Intuition (Bergson)
0.00230813642387 	 Duration, as defined by Bergson, then is a unity a
0.0445528875643 	 In ''Laughter: An Essay on the Meaning of the Comi
0.180818169777 	 From his first publications, Bergson's philosophy 
0.0617834627539 	 Alfred North Whitehead acknowledged Bergson's infl
0.0 	 Many writers of the early 20th century criticized 
0.0214383679182 	 The Vatican accused Bergson of pantheism, while fr
0.250692964314 	 Charles Sanders Peirce took strong exception to th
0.0

In [78]:
len(depth_scores)

26

In [79]:
depth_scores

[0.019906217161572148,
 0.011487118532526375,
 0.0,
 0.030074307398782918,
 0.0007105968629010917,
 0.0,
 0.003008544358082399,
 0.041334680720991557,
 0.0,
 0.0023081364238749513,
 0.044552887564330179,
 0.18081816977731835,
 0.061783462753910667,
 0.0,
 0.021438367918238721,
 0.2506929643140392,
 0.047569572997745668,
 0.0,
 0.066914528805290252,
 0.0,
 0.035797924015227189,
 0.70221114067784252,
 0.30441719966682523,
 0.17900752519158125,
 0.0,
 0.10687863467625447]

In [80]:
for i in range(len(coherence_scores)):
    print(coherence_scores[i], biographies_paragraphs[bio_index][i + window_size][:100])

0.94645787252 Bergson considers the appearance of novelty as a result of pure undetermined creation, instead of as
0.963296069778 Criticizing Kant's theory of knowledge exposed in the ''Critique of Pure Reason'' and his conception
0.986270306843 ''The Creative Evolution'' in particular attempted to think through the continuous creation of life,
0.938635427681 Henri Bergsons ''Lebensphilosophie'' (Philosophy of Life) can be seen as a response to the mechanist
0.949727969591 Bergson regarded planning beforehand for the future as impossible, since time itself unravels unfore
0.951149163317 Duration (philosophy)
0.945132074601 The foundation of Henri Bergsons philosophy, his theory of Duration, he discovered when trying to im
0.921552879783 Kant believed that free will could only exist outside of time and space, that we could therefore not
0.974625957692 Intuition (Bergson)
0.970009684844 Duration, as defined by Bergson, then is a unity and a multiplicity, but, being mobile, it cannot be
0

In [100]:
bios[47][0][1]

[('Summary',
  'Henri-Louis Bergson (; 18 October 1859 – 4 January 1941) was a major French philosopher, influential especially in the first half of the 20th century. Bergson convinced many thinkers that the processes of immediate experience and intuition are more significant than abstract rationalism and science for understanding reality.\n\nHe was awarded the 1927 Nobel Prize in Literature "in recognition of his rich and vitalizing ideas and the brilliant skill with which they have been presented". In 1930 France awarded him its highest honour, the Grand-Croix de la Legion d\'honneur.\n\n'),
 ('Philosophy',
  'Bergson rejected what he saw as the overly mechanistic predominant view of causality (as expressed in, say, finalism). He argued that we must allow space for free will to unfold in an autonomous and unpredictable fashion. While Kant saw free will as something beyond time and space and therefore ultimately a matter of faith, Bergson attempted to redefine the modern conceptions o

In [58]:
len(biographies_paragraphs[1])

17